In [1]:
from bluepy.v2 import Simulation, Circuit, Cell

import efel

from bluepy.v2.enums import Synapse

from os import path

import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import pickle

import h5py

In [ ]:
#https://bbpteam.epfl.ch/documentation/projects/BGLibPy/latest/installation.html

In [10]:
#!module load py-bglibpy/4.4.10

In [ ]:
#!pip install -i 'https://bbpteam.epfl.ch/repository/devpi/bbprelman/dev/+simple/' bglibpy[bbp]

In [2]:
import bglibpy

In [3]:
blueconfig = '/gpfs/bbp.cscs.ch/project/proj34/sh_ngvm/column/test_err/BlueConfig'

In [4]:
GIDs = [162241,162344]

In [5]:
cortex_circuit = Circuit(blueconfig)

print(cortex_circuit.cells.get(group=GIDs, properties=[Cell.MTYPE, Cell.ETYPE, Cell.SYNAPSE_CLASS, Cell.REGION]), "\n")

Have keys ({'FlushBufferScalar'}) that are not defined for section: Run


         mtype  etype synapse_class region
162241  L5_NGC  cSTUT           INH     L5
162344   L5_MC    bAC           INH     L5 



In [6]:
print(f"Number of GIDs[0] -> GIDs[1] synapses: {len(cortex_circuit.connectome.pair_synapses(pre_gid=GIDs[0], post_gid=GIDs[1]))} \
        \nNumber of GIDs[0] <- GIDs[1] synapses: {len(cortex_circuit.connectome.pair_synapses(pre_gid=GIDs[1], post_gid=GIDs[0]))}")

Number of GIDs[0] -> GIDs[1] synapses: 0         
Number of GIDs[0] <- GIDs[1] synapses: 0


In [35]:
#cortex_circuit.connectome.afferent_gids(161210)


In [7]:
ssim = bglibpy.SSim(blueconfig)

Have keys ({'FlushBufferScalar'}) that are not defined for section: Run


In [ ]:
ssim.instantiate_gids(GIDs)

In [2]:
tenpercent_gids_f = '/gpfs/bbp.cscs.ch/project/proj34/metabolismndam/in_data/mc2_tenpercent_allLayers_EXCandINH_gids.txt'
tenpercentgids = np.loadtxt(tenpercent_gids_f)

In [5]:
circuit_f = '/gpfs/bbp.cscs.ch/project/proj64/circuits/O1.v6a/20181207/CircuitConfig'

In [6]:
percent_gids = '10'

def get_cells(circuit_f,percent_gids,tenpercent_gids_f):
    c = Circuit(circuit_f)
    mc2_cells = c.cells.get({'$target': 'mc2_Column'}, properties=[Cell.X, Cell.Y, Cell.Z,Cell.SYNAPSE_CLASS,Cell.LAYER,Cell.ETYPE,Cell.MTYPE])
    mc2gidsids = {k: v for k, v in enumerate(mc2_cells.index)}

    if percent_gids == '1':
        print("work with 1% of cells")
        #subset_percent_gids = np.loadtxt(onepercent_gids_f)
    elif percent_gids == '10':
        print("work with 10% of cells")
        subset_percent_gids = np.loadtxt(tenpercent_gids_f)
    else:
        print("check percent_gids")

    subset_cells = mc2_cells.loc[subset_percent_gids,:]
    subset_cells['etype_mtype'] = subset_cells['etype'].astype(str) + "_" + subset_cells['mtype'].astype(str)
    subset_cells.index = subset_cells.index.astype(int)
    subset_cells = subset_cells.reset_index()
    subset_cells = subset_cells.rename(columns={'index':'c_gid'})

    return subset_cells


In [8]:
cells = get_cells(circuit_f,percent_gids,tenpercent_gids_f)

work with 10% of cells


In [10]:
cells.loc[cells['mtype']=='L5_TPC:A',:].head()

,c_gid,x,y,z,synapse_class,layer,etype,mtype,etype_mtype
826,161210,541.464089,1144.022761,499.611806,EXC,5,cADpyr,L5_TPC:A,cADpyr_L5_TPC:A
829,159109,415.282784,721.147620,746.057017,EXC,5,cADpyr,L5_TPC:A,cADpyr_L5_TPC:A
830,150345,304.763453,759.806697,513.226378,EXC,5,cADpyr,L5_TPC:A,cADpyr_L5_TPC:A
833,151703,391.931512,867.328447,407.668793,EXC,5,cADpyr,L5_TPC:A,cADpyr_L5_TPC:A
836,149046,454.773893,1160.618643,749.035807,EXC,5,cADpyr,L5_TPC:A,cADpyr_L5_TPC:A


In [13]:
c = Circuit(circuit_f)
mc2_cells = c.cells.get({'$target': 'mc2_Column'}, properties=[Cell.X, Cell.Y, Cell.Z,Cell.SYNAPSE_CLASS,Cell.LAYER,Cell.ETYPE,Cell.MTYPE])
mc2gidsids = {k: v for k, v in enumerate(mc2_cells.index)}

In [15]:
#Target Cell onecellL5TPCA
#{
#a161210
#}

mc2_cells.loc[mc2_cells.index==161210]

,x,y,z,synapse_class,layer,etype,mtype
161210,541.464089,1144.022761,499.611806,EXC,5,cADpyr,L5_TPC:A


In [ ]:
#/gpfs/bbp.cscs.ch/project/proj34/sh_ngvm/column/diff_pulses/onecell

In [ ]:
#/gpfs/bbp.cscs.ch/project/proj34/camps/singlecell/FInmore.py

def main(cellpath,amp):
    from man_optim import OptCell
    import efel
    import numpy as np
    import pandas as pd

    efel.setThreshold(-30)
    features = ['AP_amplitude','voltage_base','AP_duration_half_width',"AHP_depth", "AHP_time_from_peak","AP_height","APlast_amp","mean_frequency","ISI_CV","ISI_log_slope","time_to_first_spike","inv_last_ISI","inv_first_ISI","inv_second_ISI","Spikecount"]

    cell = OptCell(template=cellpath)
    name = cellpath.split('/')[-1].split('.')[0]
    cell.clear_stims()
    cell.step_current(0,1000,amp)
    cell.initialize(1000)
    cell.run()
    cell.sim_plot()
    ephys = cell.get_ephys(start=0,end=1000)[0]
    sorted_ephys = []
    for f in features:
        try:
            sorted_ephys.append(np.mean(ephys[f]))
        except:
            sorted_ephys.append(np.nan)
    return (name, amps, sorted_ephys)


if __name__ == '__main__':
    import ipyparallel as ip
    import os
    import glob
    import pandas as pd
    import numpy as np
    client = ip.Client(profile=os.getenv('IPYTHON_PROFILE'))
    lview = client.direct_view()
    cells = glob.glob('/gpfs/bbp.cscs.ch/home/camps/custom_neurodamus/emodels_dend_std/*')
    amps = np.arange(0,0.61,0.3)
    X,Y = np.meshgrid(cells,amps)
    r = lview.map_sync(main,X.flatten(),Y.flatten())
    results = pd.DataFrame(r,columns=['name','amp','features'])
    results.to_csv('FIdata.csv')


In [ ]:
#/gpfs/bbp.cscs.ch/project/proj34/camps/singlecell/FIplots.py

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.impute import SimpleImputer
from matplotlib import cm
import seaborn as sns
import glob
import os
from scipy.optimize import curve_fit
import json
from peak_detect import spike_amps

blue1 = '#0247fe'
red1 = '#fe2712'
green1 = "#66B032"
purple1 = "#8601AF"
orange1 = '#FC600A'
alpha = 0.5
colors = [blue1,red1,blue1,green1,purple1,orange1]


def listcoltopd(listcol):
    flist = [s.strip('[]').split(',') for s in listcol]
    flist = [[np.float(f) for f in l] for l in flist]
    return pd.DataFrame(flist,index=data.index,columns=features)

cmap_inh = cm.get_cmap('Reds',11)
colors_inh = cmap_inh(0.55)
cmap_exc = cm.get_cmap('summer',11)
colors_exc = cmap_exc(0.4)

data = pd.read_csv('Figures/figdata/FIdata.csv',header=0,index_col=0)

features = ['AP_amplitude','voltage_base','AP_duration_half_width',"AHP_depth", "AHP_time_from_peak","AP_height","APlast_amp","mean_frequency","ISI_CV","ISI_log_slope","time_to_first_spike","inv_last_ISI","inv_first_ISI","inv_second_ISI","Spikecount"]

featdata = listcoltopd(data.features)
s = SimpleImputer(strategy='constant',fill_value=0)
s.fit(np.array(featdata['mean_frequency']).reshape(-1,1))
featdata['mean_frequency'] = s.transform(np.array(featdata['mean_frequency']).reshape(-1,1))
#featdata['time_to_first_spike'] **= -1 #= 1/featdata['inv_first_ISI']
# featdata = pd.DataFrame(s.transform(featdata),columns=features,index=data.index)

data = pd.concat((data,featdata),axis=1)
metypes = np.unique(data.name)

fastspiking = ['SBC','CHC'] #Parv+ cells
fsdata = data[[any([f in s for f in fastspiking]) for s in data.name]]

exdata = data[['cADpyr' in s for s in data.name]]
exmean = exdata.groupby('amp').mean()

indata = data[[not 'cADpyr' in s for s in data.name]]
inmean = indata.groupby('amp').mean()


def get_expdata(featurelist):
    dirs = glob.glob('expdata/*')
    amps = []
    fs = []

    for d in dirs:
        features = json.load(open(os.path.join(d,'features.json'),'r'))
        protocols = json.load(open(os.path.join(d,'protocols.json'),'r'))
        wanted = [p for p in protocols.keys() if 'Step' in p]
        for p in wanted:
            step = protocols[p]['stimuli']['step']['amp']
            hold = protocols[p]['stimuli']['holding']['amp']
            amp = step# + hold
            amps.append(amp)
            feat = {}
            for i in features[p]['soma.v']:
                name = i['feature']
                if name in featurelist:
                    val,std = i['val'][:2]
                    feat[name] = val
            fs.append(feat)

    df = pd.DataFrame(fs, index=amps)
    df = df[df.index < 0.9]
    return df



def get_stim_freq(freq):
    amps = []
    finaltypes = []
    for m in metypes:
        freqs = data[(data['name'] == m) & (data['mean_frequency'] >= freq)]['amp']
        if freqs.any():
            finaltypes.append(m)
            amps.append(freqs.iloc[0])
    return pd.DataFrame(amps,index=finaltypes)

def FIplots(ax,feature):
    expdata = get_expdata([feature])
    expdata = expdata.sort_index()
    for metype in metypes:
        pltdata = indata[indata['name'] == metype]
        i = pltdata['amp']
        f = pltdata[feature]
        ax.plot(i,f,color=colors_inh,alpha=0.2,linewidth=0.8,label='_nolegend_')
    for metype in metypes:
        pltdata = exdata[exdata['name'] == metype]
        i = pltdata['amp']
        f = pltdata[feature]
        ax.plot(i,f,color=colors_exc,alpha=0.1,linewidth=0.8,label='_nolegend_')
    ax.plot(inmean.index,inmean[feature],color=colors_inh,linewidth=1.5,label='Inhibitory')
    ax.plot(exmean.index,exmean[feature],color=colors_exc,linewidth=1.5,label='Excitatory')
    if not expdata.empty:
        vals = np.array(expdata[feature])
        ind = expdata.index
        ax.scatter(ind,vals,color="black",s=2,alpha=0.8,marker='1',label='Experimental')
    # ax.legend(frameon=False)
    ax.set_xlabel('Input (nA)',fontsize=12)
    sns.despine(ax=ax)

def enadata(path):
    allfiles = glob.glob(path)
    freqs = []
    data = []
    for file in allfiles:
        d = np.load(file)
        freq = int(os.path.split(file)[-1].split('_')[0])
        freqs.append(freq)
        data.append(d)
    #Voltage data
    data = np.array(data)
    vdata = data[:,6,:]

    data = data[:,:6,:]
    freqs = np.array(freqs)
    unifreqs = np.unique(freqs)
    all_means = []
    all_stds = []
    for uni in unifreqs:
        sel = data[freqs == uni]
        means = np.mean(sel,axis = 0)
        stds = np.std(sel,axis = 0)
        all_means.append(means)
        all_stds.append(stds)
    all_means = np.asarray(all_means)
    all_stds = np.asarray(all_stds)
    return all_means, all_stds, unifreqs



def enaplots(axes,path):
    all_means, all_stds,freqs = enadata(path)
    t = np.linspace(0,800,all_means.shape[2])
    order = ['ena','ek','eca','v','nai','cai']
    for j, ax in enumerate(axes):
        for i,uni in enumerate([5,20,50,100],start=2):
            col = colors[i]
            meandata = all_means[freqs==uni,j,:].flatten()
            stddata = all_stds[freqs==uni,j,:].flatten()
            ax.plot(t,meandata,label='%d Hz' % uni,color=col,alpha=0.8)
            stdup = meandata + stddata
            stddown = meandata - stddata
            filldata = np.concatenate((stdup,stddown[::-1]))
            tfilldata = np.concatenate((t,t[::-1]))
            ax.fill(tfilldata,filldata,color=col,alpha=0.2)
            sns.despine(ax=ax)
        # ax.legend(frameon=False)
        ax.set_xlabel('Time (ms)')
    # ax.set_ylabel('Na reversal potential (mV)')

def nai_example(ax_vol,ax_stim):
    vol = np.load('Figures/figdata/nai_example_cADpyr_L5TPC_800.0_v.npy')
    stim = np.load('Figures/figdata/nai_example_cADpyr_L5TPC_800.0_stim.npy')
    t = np.linspace(0,25,int(25000/0.025))
    volplus = np.zeros(len(t)-len(vol)) + vol[-1]
    stimplus = np.zeros(len(t)-len(stim))
    v = np.concatenate((vol,volplus))
    s = np.concatenate((stim,stimplus))
    ax_vol.plot(t,v,color='0.1',alpha=0.8,linewidth=0.8)
    ax_vol.plot([25,25],[-15,5],color='0.1',linewidth=0.8)
    ax_vol.text(25.5,-5,'20 mV',horizontalalignment='left',verticalalignment='center',fontsize=11)
    ax_stim.plot(t,s,color='0.1',alpha=0.8,linewidth=0.8)
    ax_stim.plot([25,25],[0.2,0.4],color='0.1',linewidth=0.8)
    ax_stim.text(25.5,0.3,'0.2 nA',horizontalalignment='left',verticalalignment='center',fontsize=11)
    for s in ['top','bottom','right','left']:
        ax_vol.spines[s].set_visible(False)
        ax_stim.spines[s].set_visible(False)
    for ax in (ax_vol,ax_stim):
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.set_yticks([])
        ax.set_yticklabels([])
    ax_vol.text(0.5,55, '15 Hz',fontsize = 11   )

def alpha(t, tau,amp):
    return amp*t/tau * np.exp(1-t/tau)

#Difference with experimental data might be due to sustained NMDA activation
#500 ms at 15 Hz
def nai_plot(ax):
    allfiles = glob.glob('Figures/figdata/naidata/*')
    cells = [os.path.split(f)[-1].split('.')[0] for f in allfiles]
    data = []
    for file in allfiles:
        filedata = np.load(file)
        data.append(filedata)

    t = np.linspace(0,25,len(data[0]))
    data = np.array(data)
    for d in data:
        ax.plot(t,d,color='0.7',alpha=0.4,linewidth=1.4)
    means = np.mean(data,axis=0)
    popt,pcov = curve_fit(alpha,t,means-10,bounds=[[0,0],[5,5]])
    ax.plot(t,means, color='orange',alpha=0.8,linewidth=2.6)
    ax.plot([25,25],[12.5,13.5],color='0.1',linewidth=0.8)
    ax.text(25.5,13,'1 mM',horizontalalignment='left',verticalalignment='center',fontsize=11)
    ax.plot([23,25],[12.5,12.5],color='0.1',linewidth=0.8)
    ax.text(24,12,'2 s',horizontalalignment='center',verticalalignment='top',fontsize=11)
    b = alpha(t,*popt) + 10
    tau =popt[0]
    ax.plot(t,b,'--',color='black',linewidth=0.8)
    ax.text(5,12,'τ = %.0f ms' %(tau*1000),fontsize=11)
    # ax.set_xlabel('Time (s)')
    # ax.set_ylabel('Intracellular [Na] (mM)')
    for s in ['top','bottom','right','left']:
        ax.spines[s].set_visible(False)
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_yticklabels([])


def main():
    fig = plt.figure()
    gs = fig.add_gridspec(3,1,height_ratios=[3,1,6])
    ax_trace = fig.add_subplot(gs[0,0])
    ax_stim = fig.add_subplot(gs[1,0])
    ax_fi = fig.add_subplot(gs[2,0])
    plt.text(0.12,0.9,'1)',transform=fig.transFigure)
    plt.text(0.12,0.62,'2)',transform=fig.transFigure)
    plt.text(0.12,0.45,'3)',transform=fig.transFigure)
    nai_example(ax_trace,ax_stim)
    nai_plot(ax_fi)
    # FIplots(ax_fi,"mean_frequency")
    # ax_fi.set_ylabel('Frequency (Hz)')
    plt.show()

def FI():
    fig = plt.figure(figsize=(14,7))
    gs = fig.add_gridspec(2,3,wspace=0.28,hspace = 0.1)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[1,0],sharex=ax1)
    ax3 = fig.add_subplot(gs[0,1])
    ax4 = fig.add_subplot(gs[1,1],sharex=ax3)
    ax5 = fig.add_subplot(gs[0,2])
    ax6 = fig.add_subplot(gs[1,2],sharex=ax5)
    FIplots(ax3,'mean_frequency')
    ax3.set_ylabel('Frequency (Hz)',fontsize=12)
    FIplots(ax2,'AP_amplitude')
    ax2.set_ylabel('AP amplitude (mV)',fontsize=12)
    FIplots(ax1,'Spikecount')
    ax1.set_ylabel('Spikecount',fontsize=12)
    FIplots(ax4,'time_to_first_spike')
    ax4.set_ylabel('Time to 1st spike (ms)',fontsize=12)
    FIplots(ax5,'inv_first_ISI')
    ax5.set_ylabel('1/1st ISI (Hz)',fontsize=12)
    FIplots(ax6,'inv_second_ISI')
    ax6.set_ylabel('1/2nd ISI (Hz)',fontsize=12)
    handles,labels = ax6.get_legend_handles_labels()
    ax1.text(0.05,0.95,'A',transform=ax1.transAxes,fontweight='bold',fontsize=13)
    ax2.text(0.05,0.95,'B',transform=ax2.transAxes,fontweight='bold',fontsize=13)
    ax3.text(0.05,0.95,'C',transform=ax3.transAxes,fontweight='bold',fontsize=13)
    ax4.text(0.05,0.95,'D',transform=ax4.transAxes,fontweight='bold',fontsize=13)
    ax5.text(0.05,0.95,'E',transform=ax5.transAxes,fontweight='bold',fontsize=13)
    ax6.text(0.05,0.95,'F',transform=ax6.transAxes,fontweight='bold',fontsize=13)
    
    FIplots(ax3,'mean_frequency')
    ax3.set_ylabel('Frequency (Hz)',fontsize=12)
    FIplots(ax2,'AP_amplitude')
    ax2.set_ylabel('AP amplitude (mV)',fontsize=12)
    FIplots(ax1,'Spikecount')
    ax1.set_ylabel('Spikecount',fontsize=12)
    FIplots(ax4,'time_to_first_spike')
    ax4.set_ylabel('Time to 1st spike (ms)',fontsize=12)
    FIplots(ax5,'inv_first_ISI')
    ax5.set_ylabel('1/1st ISI (Hz)',fontsize=12)
    FIplots(ax6,'inv_second_ISI')
    ax6.set_ylabel('1/2nd ISI (Hz)',fontsize=12)
    handles,labels = ax6.get_legend_handles_labels()
    ax1.text(0.05,0.95,'A',transform=ax1.transAxes,fontweight='bold',fontsize=13)
    ax2.text(0.05,0.95,'B',transform=ax2.transAxes,fontweight='bold',fontsize=13)
    ax3.text(0.05,0.95,'C',transform=ax3.transAxes,fontweight='bold',fontsize=13)
    ax4.text(0.05,0.95,'D',transform=ax4.transAxes,fontweight='bold',fontsize=13)
    ax5.text(0.05,0.95,'E',transform=ax5.transAxes,fontweight='bold',fontsize=13)
    ax6.text(0.05,0.95,'F',transform=ax6.transAxes,fontweight='bold',fontsize=13)
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax3.get_xticklabels(), visible=False)
    plt.setp(ax5.get_xticklabels(), visible=False)
    ax1.set_xlabel('')
    ax3.set_xlabel('')
    ax5.set_xlabel('')
    fig.legend(handles,labels,loc='upper right',frameon=False)
    plt.show()

def syn_plot():
    fig = plt.figure(figsize=(14,8))
    gs = fig.add_gridspec(5,3,wspace=0.28,hspace=0.35,height_ratios=[1,1,0.1,1,1])
    axes = [fig.add_subplot(gs[i,j]) for i in [0,3] for j in [0,1,2]]
    axes1 = [fig.add_subplot(gs[i,j]) for i in [1,4] for j in [0,1,2] ]
    enaplots(axes,'Figures/figdata/syndata/*')
    enaplots(axes1,'Figures/figdata/syndata_ex/*')
    for i, (l,yl) in enumerate(zip(['A','B','C','G','H','I'],['[Na]$\mathregular{_i}$ (mM)','[K]$\mathregular{_i}$ (mM)','[Ca]$\mathregular{_i}$ (mM)','$\mathregular{E_{Na}}$ (mV)','$\mathregular{E_{K}}$ (mV)','$\mathregular{E_{Ca}}$ (mV)'])):
        axes[i].text(0.05,1.05,l,transform=axes[i].transAxes,fontweight='bold',fontsize=13)
        axes[i].set_ylabel(yl,fontsize=12)
        axes[i].set_xlabel('')
        plt.setp(axes[i].get_xticklabels(),visible=False)

    for i, (l,yl) in enumerate(zip(['D','E','F','J','K','L'],['[Na]$\mathregular{_i}$ (mM)','[K]$\mathregular{_i}$ (mM)','[Ca]$\mathregular{_i}$ (mM)','$\mathregular{E_{Na}}$ (mV)','$\mathregular{E_{K}}$ (mV)','$\mathregular{E_{Ca}}$ (mV)'])):
        axes1[i].text(0.05,1.05,l,transform=axes1[i].transAxes,fontweight='bold',fontsize=13)
        axes1[i].set_ylabel(yl,fontsize=12)
    handles,labels = axes[0].get_legend_handles_labels()
    fig.legend(handles,labels,loc='center right',frameon=False)
    plt.show()

if __name__ == '__main__':
    syn_plot()
    
    # hzdata = get_stim_freq(15)
    # hzdata.to_csv('Figures/figdata/amps_15_Hz.csv')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.impute import SimpleImputer
from matplotlib import cm
import seaborn as sns
import glob
import os
from scipy.optimize import curve_fit
import json

import sys
sys.path.append('/gpfs/bbp.cscs.ch/project/proj34/camps/singlecell')

from peak_detect import spike_amps # /gpfs/bbp.cscs.ch/project/proj34/camps/singlecell

In [3]:
blue1 = '#0247fe'
red1 = '#fe2712'
green1 = "#66B032"
purple1 = "#8601AF"
orange1 = '#FC600A'
alpha = 0.5
colors = [blue1,red1,blue1,green1,purple1,orange1]


def listcoltopd(listcol):
    flist = [s.strip('[]').split(',') for s in listcol]
    flist = [[np.float(f) for f in l] for l in flist]
    return pd.DataFrame(flist,index=data.index,columns=features)

cmap_inh = cm.get_cmap('Reds',11)
colors_inh = cmap_inh(0.55)
cmap_exc = cm.get_cmap('summer',11)
colors_exc = cmap_exc(0.4)

#data = pd.read_csv('Figures/figdata/FIdata.csv',header=0,index_col=0)
data = pd.read_csv('/gpfs/bbp.cscs.ch/project/proj34/camps/FI/cADpyr_L5TPC.csv',header=0,index_col=0)


In [4]:
data.head()

,AP_amplitude,voltage_base,AP_duration_half_width,AHP_depth,AHP_time_from_peak,AP_height,APlast_amp,mean_frequency,ISI_CV,ISI_log_slope,time_to_first_spike,inv_last_ISI,inv_first_ISI,inv_second_ISI,Spikecount
0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0
0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0
0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0
0.09,78.922489,NaN,0.900,NaN,16.433333,37.661962,79.07276,3.146633,NaN,NaN,273.4,2.935134,2.947244,2.935134,3.0
0.12,78.706591,NaN,0.925,NaN,15.875000,37.411262,78.68229,4.712535,0.00156,0.003183,169.9,4.407228,4.432624,4.416961,4.0


In [5]:
features = ['AP_amplitude','voltage_base','AP_duration_half_width',"AHP_depth", "AHP_time_from_peak","AP_height","APlast_amp","mean_frequency","ISI_CV","ISI_log_slope","time_to_first_spike","inv_last_ISI","inv_first_ISI","inv_second_ISI","Spikecount"]

featdata = listcoltopd(data.features)
s = SimpleImputer(strategy='constant',fill_value=0)
s.fit(np.array(featdata['mean_frequency']).reshape(-1,1))
featdata['mean_frequency'] = s.transform(np.array(featdata['mean_frequency']).reshape(-1,1))
#featdata['time_to_first_spike'] **= -1 #= 1/featdata['inv_first_ISI']
# featdata = pd.DataFrame(s.transform(featdata),columns=features,index=data.index)

data = pd.concat((data,featdata),axis=1)
metypes = np.unique(data.name)


AttributeError: 'DataFrame' object has no attribute 'features'

In [ ]:

fastspiking = ['SBC','CHC'] #Parv+ cells
fsdata = data[[any([f in s for f in fastspiking]) for s in data.name]]

exdata = data[['cADpyr' in s for s in data.name]]
exmean = exdata.groupby('amp').mean()

indata = data[[not 'cADpyr' in s for s in data.name]]
inmean = indata.groupby('amp').mean()

